In [1]:
import requests
import re
from bs4 import BeautifulSoup
import html5lib
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('final_sub_list.csv')

In [3]:
# The extracted subsidiaries url is incomplete, so the following function make the full URL
def join_url(url):
    return 'http://www.sec.gov'+url

In [4]:
# Aplly the function
df['EX21'] = df['EX21'].apply(join_url)

In [5]:
df.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'],inplace=True)
df

,Company Name,Form Type,CIK,Date Filed,URL,EX21
0,AFLAC INC,10-K,4977,20190225,http://www.sec.gov/Archives/edgar/data/4977/00...,http://www.sec.gov/Archives/edgar/data/4977/00...
1,ALIMERA SCIENCES INC,10-K,1267602,20190225,http://www.sec.gov/Archives/edgar/data/1267602...,http://www.sec.gov/Archives/edgar/data/1267602...
2,AMERICAN AIRLINES INC,10-K,4515,20190225,http://www.sec.gov/Archives/edgar/data/4515/00...,http://www.sec.gov/Archives/edgar/data/4515/00...
3,AMERICAN STATES WATER CO,10-K,1056903,20190225,http://www.sec.gov/Archives/edgar/data/1056903...,http://www.sec.gov/Archives/edgar/data/92116/0...
4,ARMSTRONG WORLD INDUSTRIES INC,10-K,7431,20190225,http://www.sec.gov/Archives/edgar/data/7431/00...,http://www.sec.gov/Archives/edgar/data/7431/00...
...,...,...,...,...,...,...
4055,ALJ REGIONAL HOLDINGS INC,10-K,1438731,20191223,http://www.sec.gov/Archives/edgar/data/1438731...,http://www.sec.gov/Archives/edgar/data/1438731...
4056,BARNWELL INDUSTRIES INC,10-K,10048,20191223,http://www.sec.gov/Archives/edgar/data/10048/0...,http://www.sec.gov/Archives/edgar/data/10048/0...
4057,Corning Natural Gas Holding Corp,10-K,1582244,20191223,http://www.sec.gov/Archives/edgar/data/1582244...,http://www.sec.gov/Archives/edgar/data/1582244...
4058,INNOVATIVE SOLUTIONS & SUPPORT INC,10-K,836690,20191223,http://www.sec.gov/Archives/edgar/data/836690/...,http://www.sec.gov/Archives/edgar/data/836690/...


In [10]:
def download_all_files(row):
    url = row['EX21']
    try:
        data=pd.read_html(url)
        if len(data)>1:
            data = pd.concat(d for d in data)
            data = data.dropna(axis=0,how='all')
            data = data.dropna(axis=1,how='all')
        else:
            data = data[0]
            data = data.dropna(axis=0,how='all')
            data = data.dropna(axis=1,how='all')

        data.insert(loc=0, column='cik', value=row['CIK'])
        data.insert(loc=1, column='cname', value=row['Company Name'])
        data.to_csv('democsv.csv', mode='a', index=False, header=None)
    except:
        txt = requests.get(url).content
        soup = BeautifulSoup(txt, 'html.parser')
        data = []
        for ss in soup.stripped_strings:
            data.append(ss)
        data = pd.DataFrame(data)
        data.insert(loc=0, column='cik', value=row['CIK'])
        data.insert(loc=1, column='cname', value=row['Company Name'])
        data.to_csv('democsv.csv', mode='a', index=False, header=None)
        

In [11]:
# Download subsidiaries of first 100 companies
df[:100].apply(download_all_files, axis=1)

0     None
1     None
2     None
3     None
4     None
      ... 
95    None
96    None
97    None
98    None
99    None
Length: 100, dtype: object